## Create Sampled Dataset of PC Urban


In [1]:
import argparse
import os
import yaml
import numpy as np
import numpy.linalg as la
import open3d as o3d
import pandas as pd
from matplotlib import pyplot as plt
from tqdm.auto import tqdm, trange
from tqdm.contrib.concurrent import process_map, thread_map
from matplotlib import cm
from functools import partial
import concurrent.futures
import random

from sphere import Sphere
from dh_grid import DHGrid
from laserscan import SemLaserScan

import os
import time
import matplotlib.pyplot as plt
import numpy as np
import json
import pprint
from os.path import join
import glob
import cv2
import open3d as o3d


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# dataroot = '/media/berlukas/Data/jd/camera lidar semantic'
dataroot = '/media/berlukas/Data/data/datasets/s2ae/pc urban/'
segments = os.listdir(dataroot)

print(f'Found {len(segments)} elements in the folder.')

Found 2 elements in the folder.


In [13]:
for segment in segments:
    cloud_path = f'{dataroot}/{segment}/PLY'
    label_path = f'{dataroot}/{segment}/Label'
    lidar_files = {}
    sensor_path = f'{root_path}/{lidar}/*.npz'                
    lidar_files[lidar] = sorted(glob.glob(sensor_path))
        print(f'Found {len(lidar_files[lidar])} clouds for {lidar}.')

Found 1 clouds for cam_front_center.
Found 0 clouds for cam_front_left.
Found 0 clouds for cam_front_right.
Found 0 clouds for cam_rear_center.
Found 0 clouds for cam_side_left.
Found 0 clouds for cam_side_right.
Found 1 clouds for cam_front_center.
Found 0 clouds for cam_front_left.
Found 0 clouds for cam_front_right.
Found 0 clouds for cam_rear_center.
Found 0 clouds for cam_side_left.
Found 0 clouds for cam_side_right.
Found 1 clouds for cam_front_center.
Found 0 clouds for cam_front_left.
Found 0 clouds for cam_front_right.
Found 0 clouds for cam_rear_center.
Found 0 clouds for cam_side_left.
Found 0 clouds for cam_side_right.
Found 1 clouds for cam_front_center.
Found 0 clouds for cam_front_left.
Found 0 clouds for cam_front_right.
Found 0 clouds for cam_rear_center.
Found 0 clouds for cam_side_left.
Found 0 clouds for cam_side_right.
Found 1 clouds for cam_front_center.
Found 0 clouds for cam_front_left.
Found 0 clouds for cam_front_right.
Found 0 clouds for cam_rear_center.
Foun

In [12]:
def undistort_image(image, cam_name):
    if cam_name in ['front_left', 'front_center', \
                    'front_right', 'side_left', \
                    'side_right', 'rear_center']:
        # get parameters from config file
        intr_mat_undist = \
                  np.asarray(config['cameras'][cam_name]['CamMatrix'])
        intr_mat_dist = \
                  np.asarray(config['cameras'][cam_name]['CamMatrixOriginal'])
        dist_parms = \
                  np.asarray(config['cameras'][cam_name]['Distortion'])
        lens = config['cameras'][cam_name]['Lens']
        
        if (lens == 'Fisheye'):
            return cv2.fisheye.undistortImage(image, intr_mat_dist,\
                                      D=dist_parms, Knew=intr_mat_undist)
        elif (lens == 'Telecam'):
            return cv2.undistort(image, intr_mat_dist, \
                      distCoeffs=dist_parms, newCameraMatrix=intr_mat_undist)
        else:
            return image
    else:
        return image

def extract_semantic_file_name_from_image_file_name(file_name_image):
    file_name_semantic_label = file_name_image.split('/')
    file_name_semantic_label = file_name_semantic_label[-1].split('.')[0]    
    file_name_semantic_label = file_name_semantic_label.split('_')
    file_name_semantic_label = file_name_semantic_label[0] + '_' + \
                  'label_' + \
                  file_name_semantic_label[2] + '_' + \
                  file_name_semantic_label[3] + '.png'    
    return file_name_semantic_label

def create_open3d_pc(lidar, cam_image=None):
    # create open3d point cloud
    pcd = o3d.geometry.PointCloud()
    
    # assign point coordinates
    pcd.points = o3d.utility.Vector3dVector(lidar['points'])
    
    # assign colours
    if cam_image is None:
        median_reflectance = np.median(lidar['reflectance'])
        colours = colours_from_reflectances(lidar['reflectance']) / (median_reflectance * 5)
        
        # clip colours for visualisation on a white background
        colours = np.clip(colours, 0, 0.75)
    else:
        rows = (lidar['row'] + 0.5).astype(np.int)
        cols = (lidar['col'] + 0.5).astype(np.int)
        colours = cam_image[rows, cols, :] / 255.0
        
    pcd.colors = o3d.utility.Vector3dVector(colours)    
    return pcd

def rgb_to_hex(rgb):
    return '#%02x%02x%02x' % rgb

def map_name_to_label(name):
    if name == 'Car 1':
        return 0
    if name == 'Car 2':
        return 1
    if name == 'Car 3':
        return 2
    if name == 'Car 4':
        return 3    
    if name == 'Bicycle 1':
        return 4
    if name == 'Bicycle 2':
        return 5
    if name == 'Bicycle 3':
        return 6
    if name == 'Bicycle 4':
        return 7    
    if name == 'Pedestrian 1':
        return 8
    if name == 'Pedestrian 2':
        return 9
    if name == 'Pedestrian 3':
        return 10
    if name == 'Truck 1':
        return 11
    if name == 'Truck 2':
        return 12
    if name == 'Truck 3':
        return 13    
    if name == 'Small vehicles 1':
        return 14
    if name == 'Small vehicles 2':
        return 15
    if name == 'Small vehicles 3':
        return 16    
    if name == 'Traffic signal 1':
        return 17
    if name == 'Traffic signal 2':
        return 18
    if name == 'Traffic signal 3':
        return 19    
    if name == 'Traffic sign 1':
        return 20
    if name == 'Traffic sign 2':
        return 21
    if name == 'Traffic sign 3':
        return 22
    if name == 'Utility vehicle 1':
        return 23
    if name == 'Utility vehicle 2':
        return 24    
    if name == 'Sidebars':
        return 25    
    if name == 'Speed bumper':
        return 26    
    if name == 'Curbstone':
        return 27    
    if name == 'Solid line':
        return 28    
    if name == 'Irrelevant signs':
        return 29    
    if name == 'Road blocks':
        return 30    
    if name == 'Tractor':
        return 31
    if name == 'Non-drivable street':
        return 32
    if name == 'Zebra crossing':
        return 33
    if name == 'Obstacles / trash':
        return 34
    if name == 'Poles':
        return 35
    if name == 'RD restricted area':
        return 36
    if name == 'Animals':
        return 37
    if name == 'Grid structure':
        return 38
    if name == 'Signal corpus':
        return 39
    if name == 'Drivable cobblestone':
        return 40
    if name == 'Electronic traffic':
        return 41    
    if name == 'Slow drive area':
        return 42
    if name == 'Nature object':
        return 43
    if name == 'Parking area':
        return 44
    if name == 'Sidewalk':
        return 45
    if name == 'Ego car':
        return 46
    if name == 'Painted driv. instr.':
        return 47
    if name == 'Traffic guide obj.':
        return 48
    if name == 'Dashed line':
        return 49    
    if name == 'RD normal street':
        return 50
    if name == 'Sky':
        return 51
    if name == 'Buildings':
        return 52
    if name == 'Blurred area':
        return 53
    if name == 'Rain dirt':
        return 54
    print(f'Have an unknown name: {name} !!!')
    return 0
    
def map_colors_to_label(colors):
    labels_list = []
    for label in colors:
        hex_color = rgb_to_hex(tuple(label))
        if hex_color not in class_list.keys():
            # undefined category
            # print(f'Have an unknown color {hex_color}')
            labels_list.append(0)
        else:
            labels_list.append(map_name_to_label(class_list[hex_color]))
    return np.array(labels_list)

def get_pointcloud_at(lidar, semantic_image):           
    # xyz and intensity
    points = lidar['points']
    intensity = lidar['reflectance']
    
    # semantic labels from the image
    rows = (lidar['row'] + 0.5).astype(int)
    cols = (lidar['col'] + 0.5).astype(int)
    colors = semantic_image[rows, cols, :]
    labels = map_colors_to_label(colors)
    
    pc = np.column_stack((points, intensity, labels))
    return pc

  
n_sensors = len(sensors)
all_sem_clouds = []
for sensor in sensors:
    print(f'Processing files for sensor: {sensor}')
    for file_name_lidar in tqdm(lidar_files[sensor]):
        lidar = np.load(file_name_lidar)
        
        seq_name = file_name_lidar.split('/')[-4]        
        file_name_semantic_label = extract_semantic_file_name_from_image_file_name(file_name_lidar)
        file_name_semantic_label = join(sequences, seq_name, f'label/{sensor}/', file_name_semantic_label)        
        semantic_image = cv2.imread(file_name_semantic_label, cv2.IMREAD_UNCHANGED)                        
        semantic_image = cv2.cvtColor(semantic_image, cv2.COLOR_BGR2RGB)        
        semantic_image_undistorted = undistort_image(semantic_image, sensor[4:])
        pc = get_pointcloud_at(lidar, semantic_image_undistorted)
        all_sem_clouds.append(pc)        
        
print(f'Found {len(all_sem_clouds)} pointclouds')

Processing files for sensor: cam_front_center


  0%|          | 0/1 [00:00<?, ?it/s]

Processing files for sensor: cam_front_left


0it [00:00, ?it/s]

Processing files for sensor: cam_front_right


...

Processing files for sensor: cam_side_left


0it [00:00, ?it/s]

Processing files for sensor: cam_side_right


0it [00:00, ?it/s]

Found 1 pointclouds


In [6]:
def progresser(sample_idx, grid, auto_position=True, write_safe=False, blocking=True, progress=False):
    sample = all_sem_clouds[sample_idx]
    sample_sphere = Sphere(sample)
    features = sample_sphere.sampleUsingGrid(grid)
    return features

bw = 50
grid, _ = DHGrid.CreateGrid(bw)
print(f"Loading complete. Computing features...")
# parallel
sem_idx = np.arange(0, len(all_sem_clouds))
sample_func = partial(progresser, grid=grid)
sem_features = process_map(sample_func, sem_idx, max_workers=16)            

filename = f"{export_ds}/clouds.npy"
np.save(filename, sem_features)
print(f"Wrote features to {filename}.")

filename = f'archive'
np.save(f'{export_ds}/{filename}.npy', all_sem_clouds)
print(f'Saved clouds to {export_ds}/{filename}.npy')

Loading complete. Computing features...


/tmp/ipykernel_34996/2062938013.py:13: TqdmWarning: Iterable length 4122 > 1000 but `chunksize` is not set. This may seriously degrade multiprocess performance. Set `chunksize=1` or more.
  sem_features = process_map(sample_func, sem_idx, max_workers=16)


  0%|          | 0/4122 [00:00<?, ?it/s]

Wrote features to /media/berlukas/Data/jd/camera lidar semantic/processed/clouds.npy.


/usr/lib/python3/dist-packages/numpy/lib/npyio.py:528: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.asanyarray(arr)


Saved clouds to /media/berlukas/Data/jd/camera lidar semantic/processed/archive.npy


## Visualize extracted pointclouds

In [23]:
def mapIntensityToRGB(i):
    return cm.jet(plt.Normalize(min(i), max(i))(i))

def visualizeRawPointcloud(pcl, val):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pcl[:, 0:3])
    colors = mapIntensityToRGB(val)
#     colors = scan.sem_color_lut[pcl[:,4].astype(np.int)]
    pcd.colors = o3d.utility.Vector3dVector(colors[:,0:3])
    o3d.visualization.draw_geometries([pcd])    
    
def createGrid_old(bw):
    n_grid = 2 * bw
    k = 0;
    points = np.empty([n_grid * n_grid, 2])
    for i in range(n_grid):
        for j in range(n_grid):
            points[k, 0] = (np.pi*(2*i+1))/(4*bw)
            points[k, 1] = (2*np.pi*j)/(2*bw);
            k = k + 1;
    return points
    
def convertGridToEuclidean_old(grid):
    cart_grid = np.zeros([ grid.shape[0], 3])
    cart_grid[:,0] = np.multiply(np.sin(grid[:,0]), np.cos(grid[:,1]))
    cart_grid[:,1] = np.multiply(np.sin(grid[:,0]), np.sin(grid[:,1]))
    cart_grid[:,2] = np.cos(grid[:,0])
    return cart_grid

def create_sampling_sphere(bw):
    grid = createGrid_old(bw)
    xyz_grid = convertGridToEuclidean_old(grid)
    intensities = np.zeros((xyz_grid.shape[0],1))
    sampling_grid = np.hstack((xyz_grid, np.ones((xyz_grid.shape[0], 1), dtype=xyz_grid.dtype)))
    return sampling_grid.T

In [34]:
pc = all_sem_clouds[710]
visualizeRawPointcloud(pc, pc[:, 4])

In [7]:
classes = np.unique(pc[:, 4])
print(f'we have these classes: {classes}')

we have these classes: [ 0.  1.  2.  8.  9. 20. 27. 29. 34. 35. 39. 43. 45. 48. 50. 51. 52.]


In [8]:
cur_sem_cloud = sem_features[1444]

cur_sem_cloud = np.reshape(cur_sem_cloud, (3, -1)).T
pc = create_sampling_sphere(bw)
points_xyz = pc.T[:,0:3]
points_xyzl = np.column_stack((points_xyz, cur_sem_cloud[:,2]))

visualizeRawPointcloud(points_xyzl, points_xyzl[:, 3])